In [70]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector

import numpy as np
import pandas as pd

import time 
import datetime

In [71]:
class CleaningRobotAgent(Agent):
    def __init__ (self, id, model):
        super().__init__(id, model)
        
    def step(self):
        (x, y) = self.pos
        if self.model.dirty_cells[x][y] == 1:
            self.model.dirty_cells[x][y] = 0
        else:
            neighborhood = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
            (x, y) = self.random.choice(neighborhood)
            cell_content = self.model.grid.get_cell_list_contents([(x, y)])
            if not cell_content:
                self.model.grid.move_agent(self, (x, y))

In [72]:
def get_grid(model):
    grid_info = np.zeros((model.grid.width, model.grid.height))

    for x in range(model.grid.width):
        for y in range(model.grid.height):
            cell_content = model.grid.get_cell_list_contents([(x, y)])

            # Verifica la presencia de agentes
            if cell_content:
                grid_info[x][y] = 2

            # Verifica el estado de limpieza de la celda
            if not cell_content:
                grid_info[x][y] = model.dirty_cells[x][y]

    return grid_info

In [73]:
class CleaningModel(Model):
    def __init__ (self, width, height, num_vaccums, dirty_percentage):
        self.num_vaccums = num_vaccums
        self.amount_of_dirty_cells = int((width * height) * (dirty_percentage / 100))
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(width, height, False)
        self.dirty_cells = np.zeros( (width, height) )
        #Gets the position of the agents
        self.datacollector = DataCollector(
            model_reporters={"Grid": get_grid},
            agent_reporters={"Position": "pos"}
        )

        #Initializes vacuums in cell [1,1]
        for i in range(self.num_vaccums):
            agent = CleaningRobotAgent(i, self)
            self.schedule.add(agent)
            self.grid.place_agent(agent, (1,1))

        #Initializes dirty cells
        count = 0
        print(self.dirty_cells)
        while count < self.amount_of_dirty_cells:
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            if self.dirty_cells[x][y] == 0:
                self.dirty_cells[x][y] = 1
                count += 1
          
    def step(self):
        #Collects data from the agents defined in the reporters
        self.datacollector.collect(self)
        #Moves to the next step of the simulation
        self.schedule.step()

In [79]:
WIDTH = 100
HEIGHT = 100
NUM_VACCUMS = 1
DIRTY_PERCENTAGE = 90
MAX_STEPS = 100

print('Case 1 for 100 steps')
start_time = time.time()
model = CleaningModel(WIDTH, HEIGHT, NUM_VACCUMS, DIRTY_PERCENTAGE)
for i in range(MAX_STEPS):
    model.step()

#Remaining dirty cells
dirty_cells_count = np.sum(model.dirty_cells)
print("Remaining dirty cells:", dirty_cells_count)

#Print execution time
print('Execution Time:', str(datetime.timedelta(seconds=(time.time() - start_time))))

Case 1 for 100 steps
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Remaining dirty cells: 8973.0
Execution Time: 0:00:02.127341


In [83]:
WIDTH = 100
HEIGHT = 100
NUM_VACCUMS = 2
DIRTY_PERCENTAGE = 90
MAX_STEPS = 100

print('Case 2 for 100 steps')
start_time = time.time()
model = CleaningModel(WIDTH, HEIGHT, NUM_VACCUMS, DIRTY_PERCENTAGE)
for i in range(MAX_STEPS):
    model.step()

#Remaining dirty cells
dirty_cells_count = np.sum(model.dirty_cells)
print("Remaining dirty cells:", dirty_cells_count)

#Print execution time
print('Execution Time:', str(datetime.timedelta(seconds=(time.time() - start_time))))

Case 2 for 100 steps
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Remaining dirty cells: 8938.0
Execution Time: 0:00:02.210579


In [81]:
WIDTH = 100
HEIGHT = 100
NUM_VACCUMS = 1
DIRTY_PERCENTAGE = 90
MAX_STEPS = 1000

print('Case 1 for 1000 steps')
start_time = time.time()
model = CleaningModel(WIDTH, HEIGHT, NUM_VACCUMS, DIRTY_PERCENTAGE)
for i in range(MAX_STEPS):
    model.step()

#Remaining dirty cells
dirty_cells_count = np.sum(model.dirty_cells)
print("Remaining dirty cells:", dirty_cells_count)

#Print execution time
print('Execution Time:', str(datetime.timedelta(seconds=(time.time() - start_time))))


Case 1 for 1000 steps
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Remaining dirty cells: 8719.0
Execution Time: 0:00:15.458526


In [84]:
WIDTH = 100
HEIGHT = 100
NUM_VACCUMS = 2
DIRTY_PERCENTAGE = 90
MAX_STEPS = 1000

print('Case 2 for 1000 steps')
start_time = time.time()
model = CleaningModel(WIDTH, HEIGHT, NUM_VACCUMS, DIRTY_PERCENTAGE)
for i in range(MAX_STEPS):
    model.step()

#Remaining dirty cells
dirty_cells_count = np.sum(model.dirty_cells)
print("Remaining dirty cells:", dirty_cells_count)

#Print execution time
print('Execution Time:', str(datetime.timedelta(seconds=(time.time() - start_time))))


Case 2 for 1000 steps
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Remaining dirty cells: 8637.0
Execution Time: 0:00:15.811803


In [82]:
WIDTH = 100
HEIGHT = 100
NUM_VACCUMS = 1
DIRTY_PERCENTAGE = 90
MAX_STEPS = 10000

print('Case 1 for 10000 steps')
start_time = time.time()
model = CleaningModel(WIDTH, HEIGHT, NUM_VACCUMS, DIRTY_PERCENTAGE)
for i in range(MAX_STEPS):
    model.step()

#Remaining dirty cells
dirty_cells_count = np.sum(model.dirty_cells)
print("Remaining dirty cells:", dirty_cells_count)

#Print execution time
print('Execution Time:', str(datetime.timedelta(seconds=(time.time() - start_time))))

Case 1 for 10000 steps
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Remaining dirty cells: 7085.0
Execution Time: 0:02:39.802152


In [85]:
WIDTH = 100
HEIGHT = 100
NUM_VACCUMS = 2
DIRTY_PERCENTAGE = 90
MAX_STEPS = 10000

print('Case 2 for 10000 steps')
start_time = time.time()
model = CleaningModel(WIDTH, HEIGHT, NUM_VACCUMS, DIRTY_PERCENTAGE)
for i in range(MAX_STEPS):
    model.step()

#Remaining dirty cells
dirty_cells_count = np.sum(model.dirty_cells)
print("Remaining dirty cells:", dirty_cells_count)

#Print execution time
print('Execution Time:', str(datetime.timedelta(seconds=(time.time() - start_time))))

Case 2 for 10000 steps
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Remaining dirty cells: 5106.0
Execution Time: 0:02:51.185637


In [87]:
WIDTH = 100
HEIGHT = 100
NUM_VACCUMS = 1
DIRTY_PERCENTAGE = 90

print('Case 1 Steps required to clean all the cells')
start_time = time.time()
model = CleaningModel(WIDTH, HEIGHT, NUM_VACCUMS, DIRTY_PERCENTAGE)
dirty_cells_count = np.sum(model.dirty_cells)
steps = 0

while dirty_cells_count > 0:
    model.step()
    steps += 1
    dirty_cells_count = np.sum(model.dirty_cells)

#Print steps 
print("Steps required:", steps)
#Print execution time
print('Execution Time:', str(datetime.timedelta(seconds=(time.time() - start_time))))

Case 1 Steps required to clean all the cells
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Steps required: 214321
Execution Time: 1:14:56.457664


In [86]:
WIDTH = 100
HEIGHT = 100
NUM_VACCUMS = 2
DIRTY_PERCENTAGE = 90

print('Case 2 Steps required to clean all the cells')
start_time = time.time()
model = CleaningModel(WIDTH, HEIGHT, NUM_VACCUMS, DIRTY_PERCENTAGE)
dirty_cells_count = np.sum(model.dirty_cells)
steps = 0

while dirty_cells_count > 0:
    model.step()
    steps += 1
    dirty_cells_count = np.sum(model.dirty_cells)

#Print steps 
print("Steps required:", steps)
#Print execution time
print('Execution Time:', str(datetime.timedelta(seconds=(time.time() - start_time))))

Case 2 Steps required to clean all the cells
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Steps required: 142721
Execution Time: 0:38:08.953009
